In [ ]:
import cv2
import numpy as np
import glob
import time

def label():

    k = np.zeros((4,4), 'float')
    for i in range(4):
        k[i,i] = 1

    temp_label = np.zeros((1,4),'float')
    image_array1 = np.zeros((1,115200),'float')
    label_array1 = np.zeros((1,4),'float')
    count = 0
    for img in glob.glob("left/*.png"):
        image = cv2.imread(img,cv2.IMREAD_COLOR)
        roi = image[120:240, :, :]
        temp_array1 = roi.reshape(1,115200).astype(np.float32)
        image_array1 = np.vstack((image_array1, temp_array1))
        label_array1 = np.vstack((label_array1, k[0]))
        count+=1
    print('left done')
    print(count)

    for img in glob.glob("right/*.png"):
        image = cv2.imread(img,cv2.IMREAD_COLOR)
        roi = image[120:240, :, :]
        temp_array1 = roi.reshape(1,115200).astype(np.float32)
        image_array1 = np.vstack((image_array1, temp_array1))
        label_array1 = np.vstack((label_array1, k[2]))
        count+=1
    print('right done')
    print(count)

    for img in glob.glob("forward/*.png"):
        image = cv2.imread(img,cv2.IMREAD_COLOR)
        roi = image[120:240, :, :]
        temp_array1 = roi.reshape(1,115200).astype(np.float32)
        image_array1 = np.vstack((image_array1, temp_array1))
        label_array1 = np.vstack((label_array1, k[1]))
        count+=1
    print('forward done')
    print(count)

    for img in glob.glob("reverse/*.png"):
        if not img:
            break
        else:
            image = cv2.imread(img,cv2.IMREAD_COLOR)
            roi = image[120:240, :, :]
            temp_array1 = roi.reshape(1,115200).astype(np.float32)
            image_array1 = np.vstack((image_array1, temp_array1))
            label_array1 = np.vstack((label_array1, k[3]))
            count+=1
    print('reverse done')
    print(count)

    train = image_array1[1:, :]
    train_labels = label_array1[1:, :]
    
    return train, train_labels


In [ ]:
import h5py
import time
train, train_labels = label()
fileName = 'kerasTraining/training'+time.strftime("%Y%m%d-%H%M%S")+'.h5'
with h5py.File(fileName,'w') as f:
    f.create_dataset("train", data = train)
    f.create_dataset("train_labels", data = train_labels)
print('h5py file created!')

In [ ]:
''' train shape = number of images, 115200 pixels.
    train labels shape = number of images, 4.
    [1 0 0 0] = left
    [0 1 0 0] = forward
    [0 0 1 0] = right
    [0 0 0 1] = reverse/stop'''

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD,Adam
from sklearn.cross_validation import train_test_split

# Generate data
def data_loader(filename):
    with h5py.File('kerasTraining/'+filename+'.h5', 'r') as hf:
        datas = hf['train'].value
        labels = hf['train_labels'].value
    print("datas",datas.shape)
    print("labels",labels.shape)
    datas = datas / 255
    training_datas,validation_datas,training_labels,validation_labels = train_test_split(datas,labels,test_size=0.3)
    #training_labels = keras.utils.to_categorical(training_labels)
    #validation_labels = keras.utils.to_categorical(validation_labels)
    return datas, training_datas, validation_datas, training_labels, validation_labels

datas, x_train,x_test,y_train,y_test = data_loader('training20180429-212713')
print("x_train : ",x_train.shape)
print("y_train :",y_train.shape)
print("x_test : ",x_test.shape)
print("y_test : ",y_test.shape)


model = Sequential()

model.add(Dense(512, activation='relu',input_dim=115200))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='sgd',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,batch_size=50)
score = model.evaluate(x_test,y_test,batch_size=50)

In [ ]:
loss = score[0]
accuracy = score[1]
print('loss: ',loss)
print('accuracy : ',accuracy)

In [ ]:
import time
timestr = time.strftime('%Y%m%d_%H%M%S')
filename_timestr = 'kerasModel_{}.h5'.format(timestr)
model.save('KerasModels/kerasModel_{}.h5'.format(timestr))
print('keras model saved!')

In [ ]:
import json
import csv
import glob

training_data = data_loader('kerasModel_20180429_214548')


# Save parameters to json file
json_string = model.to_json()
with open('./logs/nn_params_json/nn_{}.json'.format(timestr), 'w') as new_json:
    json.dump(json_string, new_json)

# Save training results to csv log
row_params = [str(training_data)[-33:-2], filename_timestr, loss, accuracy]
with open('./logs/log_nn_training.csv','a') as log:
    log_writer = csv.writer(log)
    log_writer.writerow(row_params)